<a href="https://colab.research.google.com/github/SantiData87/yFinance/blob/main/BCRA_API_NUEVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests, pandas as pd

url = "https://api.bcra.gob.ar/estadisticas/v1/principalesvariables"
r = requests.get(url, verify=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bcra.gob.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
# Listado de variables que se pueden obtener
variables = pd.DataFrame(r.json().get('results'))
variables

""


In [6]:
# Funcion para obtener el historico de cada variable

def getValues(vId, from_, to_ ):
    url = f"https://api.bcra.gob.ar/estadisticas/v1/datosvariable/{vId}/{from_}/{to_}"
    r = requests.get(url, verify=False)
    df = pd.DataFrame(r.json().get('results')).iloc[:,-2:].set_index('fecha')
    df.index = pd.to_datetime(df.index, format='%d/%m/%Y')
    df['valor'] = df['valor'].str.replace('.','')
    df['valor'] = df['valor'].str.replace(',','.')
    df = df.astype(float)
    return df

In [7]:
# id = 27 >> IPC MENSUAL
# id = 5 >> TDC Mayorista

# Obtenemos el historico del TDC y del IPC
from_, to_ = "2010-01-01",  "2024-04-30"
tdc = getValues(5,from_, to_)
ipc_mes = getValues(27, from_, to_)
tdc

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bcra.gob.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyError: "None of ['fecha'] are in the columns"

In [ ]:
tdc_pct_change = tdc.resample('M').mean().pct_change().dropna()
tdc_ipc = pd.concat([tdc_pct_change, ipc_mes], axis=1).dropna()
tdc_ipc.columns = ['TDC','IPC']
tdc_ipc['year'] = tdc_ipc.index.year

print(f"\n\nCorrelacion entre variacion de TDC y IPC: {tdc_ipc.TDC.corr(tdc_ipc.IPC):.2f}\n\n")
display(tdc_ipc)



Correlacion entre variacion de TDC y IPC: 0.68




,TDC,IPC,year
fecha,,,
2010-02-28,0.012089,1.2,2010
2010-03-31,0.002927,1.1,2010
2010-04-30,0.003554,0.8,2010
2010-05-31,0.007142,0.7,2010
2010-06-30,0.006009,0.7,2010
...,...,...,...
2023-11-30,0.010912,12.8,2023
2023-12-31,0.814353,25.5,2023
2024-01-31,0.274686,20.6,2024
